i have two nodes for input and one node for output, there is a one hidden layer with hidden_count number of nodes(init(hidden_count, alpha)), (when i changed it from 5 to 8 it took more time and the resault got a little better _90.7 to 91.5_ and time around 5 sec to 25 sec...) 
*at the end of report i printed the points that were incorrect

Getting input

In [37]:
from math import exp
import random

file = open("train", "r")
count = 0
in_x = []
in_y = []
t = []
while 1:
 
    line = file.readline()

    if not line:
        break
    line = list(line.split())
    in_x.append(float(line[0]))
    in_y.append(float(line[1]))
    t.append(int(line[2]))
    #print(in_x[-1], in_y[-1], t[-1])
    count += 1
print(len(in_x), in_x[:10])
print(max(in_x), min(in_x), max(in_y), min(in_y))

4000 [-0.586110177649249, -1.50363417532463, -1.28089947174288, -0.023598126223051, 1.33647812419478, 0.357641911027138, -0.267087831778927, -1.25246127938213, -0.486626837029913, 1.35142920975526]
2.97797123593957 -3.04444701414054 4.94845678331942 -3.00517265841451


Initializing the first random values (between -0.5 and 0.5) and +1 is for bias value

In [26]:
def init(hidden_count, a):
    global v, w, z, alpha, bias_v, bias_w
    alpha = a
    z = [0 for i in range(hidden_count)]
    bias_v = [(random.random() - 0.5) for i in range(hidden_count)]
    v = [[(random.random() - 0.5) for i in range(hidden_count)] for j in range(2)]
    w = [(random.random() - 0.5)for i in range(hidden_count)]
    bias_w = random.random() - 0.5
init(5, 0.02)
print(v, w)

[[-0.10054693782136348, 0.08154030636444798, 0.44237808856641925, -0.3811777105096721, 0.25004901689815096], [-0.12302832259318297, 0.3879858365929586, 0.374978888415581, 0.4132414799176106, -0.33706161375952803]] [-0.018391279890557444, 0.03709925923559865, 0.1596915685433138, 0.3545132794536243, -0.197756731834846]


Activation function(sigmoid) and it's derivative

In [50]:
def f(x):
    #print(x)
    if x > 500: return 1
    if x < -500:return 0
    return 1.0 / (1.0 + exp(-x))
def df(x):
    return f(x) * (1.0 - f(x))

Update the values with one input (x,y,t)
(terminology is like the class slides)

In [55]:
def train_one(x, y, t, res = 0):
    global v, w, z, alpha, bias_v, bias_w
    
    z_in = []
    for j in range(len(z)):
        z_in.append(bias_v[j] + v[0][j] * x + v[1][j] * y)
        z[j] = f(z_in[-1])
    out = bias_w + sum([w[i] * z[i] for i in range(len(z))]) 
    out_act = f(out)
    if res : return out_act
    # returning for error and update
    delta = (t - out_act) * df(out)
    delta_w = [alpha * delta * z[i] for i in range(len(z))]
    delta_bias_w = alpha * delta
    
    delta_in = [delta * w[j] for j in range(len(z))]
    delta = [delta_in[i] * df(z_in[i]) for i in range(len(z))]
    #print(delta)
    delta_v0 = [alpha * delta[j] * x for j in range(len(z))]
    delta_v1 = [alpha * delta[j] * y for j in range(len(z))]
    delta_bias_v = [alpha * delta[j] for j in range(len(z))]
    
    for i in range(len(z)):
        w[i] += delta_w[i] 
        v[0][i] += delta_v0[i]
        v[1][i] += delta_v1[i]
        bias_v[i] += delta_bias_v[i]
        
    bias_w += delta_bias_w

In [66]:
init(5, .01)
for i in range(400):
    for j in range(4000):
        train_one(in_x[j], in_y[j], t[j])


In [54]:
print(v, w)

[[1.262067347178126, 1.7692711607656046, -1.3274107589944268, -1.887292372463022, 1.7812517385180215], [0.14378938404170624, 0.40476468367712776, -0.21530310374284073, -0.4383637496790415, 0.62858991478586]] [1.1682783625315516, 1.9418454920700423, -1.4375403623765952, -2.2135577881958195, 1.9530999034865162]


i had this crazy bug that instead of f'(x) = f(x) * (1-f(x)) it was f'(x) = x * (1-x) and i changed a lot other parts for it... 

In [67]:
file = open("test", "r")
count = 0
correct = 0
while 1:
 
    line = file.readline()

    if not line:
        break
    line = list(line.split())
    a = (float(line[0]))
    b = (float(line[1]))
    c = (int(line[2]))
    if abs(c - train_one(a,b,c,1)) < 0.4:
        correct += 1

    #print(in_x[-1], in_y[-1], t[-1])
    
    count += 1
print(correct / count)

0.915


In [68]:
file = open("test", "r")
count = 0
correct = 0
while 1:
 
    line = file.readline()

    if not line:
        break
    line = list(line.split())
    a = (float(line[0]))
    b = (float(line[1]))
    c = (int(line[2]))
    if abs(c - train_one(a,b,c,1)) < 0.4:
        correct += 1
    else:
        print(a,b)    


-0.633687809518789 0.469846539860649
0.200443114488755 -1.39076610484662
0.020529179999515 -0.741064312936821
0.052981422316654 -0.417082523824423
0.230474265066374 1.32188830412696
0.208502349528497 -1.50069896375079
0.504925712957714 1.33289141417598
0.140151446933644 -0.269437066924355
-0.025901124087941 -0.637463169792611
-0.772839077233568 2.54892115425235
0.043788091779218 -0.245058038136038
0.201886481629137 -1.52495389580395
-0.182554291108849 -0.591716871650174
-0.299854678639638 0.081549971845352
-0.748389402883773 -0.519726810513816
-0.180832995475311 -1.37166912426394
1.00480338161153 -2.24461117748409
-0.287569052828006 0.104110966761412
0.22535723175402 -1.73359793273308
-0.842592943376905 -1.40771658085872
0.081527123525265 -0.352858172528667
-0.267573699467927 -1.60862593686852
0.523300924134902 -1.22786948924675
0.403799434702408 -1.22460591754875
0.252981545294199 -1.17197595898611
0.028905774508546 -0.723527073613366
0.959271009910824 -2.68560885690102
0.087625271852